In [1]:
import warnings
import os
warnings.filterwarnings("ignore")

import numpy as np

In [2]:
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import math
from sklearn.datasets import make_spd_matrix
plt.rcParams["axes.grid"] = False
from scipy.stats import multivariate_normal
from fuzzywuzzy import fuzz

## Constant Variable

In [4]:
data_path = "liquor_data_new.csv"
flavour_columns = ["f1", "f2", "f3", "f4", "f5", "f6"]

## Initial Data

In [5]:
def load_data(data_path=data_path):
    return pd.read_csv(data_path, encoding="utf-8", sep='\t')

In [6]:
df_initial = load_data()
df_initial

,name,intl_name,brand_name,brand_intl_name,year_month,rank,score,f1,f2,f3,f4,f5,f6,flavour_tags,checkin_count,pictures,similar_brands,id,similar_brand_taste
0,上川大雪,Kamikawataisetsu,北海道,Hokkaido,202001,1,4.299969,0.516103,0.484763,0.203029,0.419481,0.276085,0.453028,|グイグイ|メロン|フレッシュ|上品|丸み|フルーティ|旨味|甘味|酸味|マスカット|ガス|...,117,https://s3-ap-northeast-1.amazonaws.com/physal...,加茂錦|而今|陸奥八仙|宮寒梅|十四代,8685,上川大雪|東洋美人|十四代|冩楽|笑四季|Ohmine Junmai|川中島|豊明|至|一白水成
1,國稀,Kunimare,北海道,Hokkaido,202001,2,4.099463,0.187750,0.304858,0.417155,0.539935,0.527491,0.357793,|辛口|蜜|スッキリ|力強い|旨味|ゴクゴク|洋梨|若い|じっくり|薄い|円やか|ちびちび|...,210,https://s3-ap-northeast-1.amazonaws.com/physal...,月山|黒牛|龍力|谷川岳|春鹿,1928,國稀|黎明|越前岬|鷹来屋|宮の雪|初孫|住吉|春鶯囀|長良川|辰泉
2,男山,Otokoyama,北海道,Hokkaido,202001,3,4.095614,0.277607,0.394435,0.327092,0.520835,0.454270,0.427476,|辛口|酸味|甘味|旨味|スッキリ|冷酒|常温|ヨーグルト|ヤクルト|熱燗|ひろがる|バラン...,397,https://s3-ap-northeast-1.amazonaws.com/physal...,春鹿|まんさくの花|酔鯨|加賀鳶|いづみ橋,2,男山|八海山|喜楽長|杉勇|神聖|大那|如空|千代むすび|七冠馬|白隠正宗
3,国士無双,Kokushimuso,北海道,Hokkaido,202001,4,4.079436,0.387998,0.256487,0.161930,0.372511,0.593511,0.516169,|辛口|洋梨|ピリリ|フルーティ|荒々しい|スパイシー|さらり|果物|酸味|さわやか|スイス...,208,https://s3-ap-northeast-1.amazonaws.com/physal...,上喜元|酔心|澤乃井|加賀鳶|司牡丹,10,国士無双|真野鶴|無想|藍の郷|あら玉|福祝|幻の瀧|高清水|きりんざん|北光正宗
4,北の錦,Kitanonishiki,北海道,Hokkaido,202001,5,4.055023,0.344262,0.393210,0.501239,0.564264,0.211910,0.335154,|とろみ|荒々しい|力強い|ほのか|薄い|トロピカル|ヨーグルト|濃厚|苦味|チーズ|旨味|...,85,https://s3-ap-northeast-1.amazonaws.com/physal...,誠鏡|原田|龍力|旦|天の戸,11,北の錦|爛漫|木戸泉|松みどり|芳水|千福|玉櫻|磐城壽|豊久仁|喜久酔
5,二世古,Niseko,北海道,Hokkaido,202001,6,4.026027,0.350896,0.474453,0.414929,0.447702,0.322182,0.418762,|栗|砂糖|旨味|しっかり|複雑|マスカット|辛口|円やか|綺麗|キリリ|甘味|どっしり|フ...,221,https://s3-ap-northeast-1.amazonaws.com/physal...,るみ子の酒|雁木|篠峯|喜楽長|会津中将,1611,弥右衛門|会津中将|遊穂|春霞|梵|高砂|花泉|二世古|鏡山|大関
6,北の誉,Kitanohomare,北海道,Hokkaido,202001,7,4.024530,0.296399,0.359640,0.255484,0.766278,0.248070,0.262324,|芳香|洋梨|淡い|クリーミー|昔ながら|熱燗|渋み|冷酒|旨味|するする|穀物|なめらか|...,67,https://s3-ap-northeast-1.amazonaws.com/physal...,松竹梅|銀嶺立山|澤乃井|英勲|千徳,7,北の誉|酔仙|緑川|富翁|雪中梅|鷗樹|白馬錦|たけのそのパンダフェスティバル|水鳥記|六歌仙
7,北の勝,Kitanokatsu,北海道,Hokkaido,202001,8,4.024530,NaN,NaN,NaN,NaN,NaN,NaN,|マイルド|フレッシュ|花|豊か|キレ|フルーティ|ガス|スッキリ|軽快|辛口|甘味|バラン...,63,https://s3-ap-northeast-1.amazonaws.com/physal...,墨廼江|加茂錦|繁枡|大信州|土佐しらぎく,1562,NaN
8,千歳鶴,Chitosetsuru,北海道,Hokkaido,202001,9,4.018903,0.281671,0.263296,0.229914,0.466118,0.636336,0.419865,|ゴクゴク|じわじわ|セメダイン|キリリ|複雑|豊か|辛口|ツン|どっしり|華やか|さらり|...,81,https://s3-ap-northeast-1.amazonaws.com/physal...,福祝|巌|〆張鶴|七本鎗|黄桜,1734,千歳鶴|吉乃川|銀盤|榮川|谷川岳|白鷹|高清水|司牡丹|〆張鶴|南
9,福司,福司,北海道,Hokkaido,202001,10,4.012568,0.253412,0.382632,0.420356,0.441749,0.481632,0.430771,|ハチミツ|穀物|ミルク|甘味|力強い|キャラメル|ガス|ふくよか|キレ|フレッシュ|優しい...,80,https://s3-ap-northeast-1.amazonaws.com/physal...,磐城寿|蒼天伝|小左衛門|房島屋|御前酒,2647,麓井|川亀|苗加屋|千代むすび|長門峡|想天坊|奥の松|出雲富士|播州一献|男山


## Data Cleaning

In [7]:
def sequel(string_1, string_2):
    if pd.isnull(string_1): return False
    return (fuzz.ratio(string_1, string_2) >= 70 or fuzz.token_sort_ratio(string_1, string_2) >= 70)

In [8]:
df_missing_f = df_initial[df_initial.f1.isnull() & df_initial.flavour_tags.notnull()]

In [9]:
df_missing_f.head()

,name,intl_name,brand_name,brand_intl_name,year_month,rank,score,f1,f2,f3,f4,f5,f6,flavour_tags,checkin_count,pictures,similar_brands,id,similar_brand_taste
7,北の勝,Kitanokatsu,北海道,Hokkaido,202001,8,4.024530,NaN,NaN,NaN,NaN,NaN,NaN,|マイルド|フレッシュ|花|豊か|キレ|フルーティ|ガス|スッキリ|軽快|辛口|甘味|バラン...,63,https://s3-ap-northeast-1.amazonaws.com/physal...,墨廼江|加茂錦|繁枡|大信州|土佐しらぎく,1562,NaN
10,大雪乃蔵,Taisetsunokura,北海道,Hokkaido,202001,11,4.011490,NaN,NaN,NaN,NaN,NaN,NaN,|丸み|優しい|スイスイ|辛口|旨味|キリリ|さわやか|フレッシュ|華やか|バランス|常温|...,50,https://s3-ap-northeast-1.amazonaws.com/physal...,若竹|臥龍梅|あさ開|谷川岳|きりんざん,3,NaN
11,風のささやき,Kazenosasayaki,北海道,Hokkaido,202001,12,4.011301,NaN,NaN,NaN,NaN,NaN,NaN,|さらり|優しい|軽快|常温|さっぱり|辛口|旨味|スッキリ|酸味,14,https://s3-ap-northeast-1.amazonaws.com/physal...,NaN,8,NaN
12,十一州,NaN,北海道,Hokkaido,202001,13,4.009420,NaN,NaN,NaN,NaN,NaN,NaN,|昔ながら|控えめ|甘味|穏やか|常温|さっぱり|上品|キリリ|苦味|軽快|余韻|フルーティ...,44,https://s3-ap-northeast-1.amazonaws.com/physal...,松みどり|聖泉|田酒|黎明|豊盃,2309,NaN
13,まる田,Maruta,北海道,Hokkaido,202001,14,4.009420,NaN,NaN,NaN,NaN,NaN,NaN,|あっさり|コク|薄い|さらり|熱燗|上品|バランス|キレ|常温|濃厚|スッキリ|酸味|しっ...,20,https://s3-ap-northeast-1.amazonaws.com/physal...,湊屋藤助|ん|大七|天領|義侠,1275,NaN


In [10]:
def filling_data(df, df_missing):
    df_new = df.copy()
    
    for idx, row in df_missing.iterrows():
        flavour_tags = row['flavour_tags']
        group_data = df_new[df_new['flavour_tags'].apply(lambda x: sequel(x, flavour_tags))]
        mean_f1 = group_data['f1'].mean()
        if pd.isnull(mean_f1): continue
            
        mean_f2 = group_data['f2'].mean()
        mean_f3 = group_data['f3'].mean()
        mean_f4 = group_data['f4'].mean()      
        mean_f5 = group_data['f5'].mean()
        mean_f6 = group_data['f6'].mean()
        
        group_similar_taste = []
        similar_brand_taste = group_data['similar_brand_taste'].values
        
#         print(similar_brand_taste)
#         break
        for sbt in similar_brand_taste:
            if pd.notnull(sbt):
                for it in sbt.split("|"):
                    if it not in group_similar_taste: group_similar_taste.append(it)
        
        df_new.at[idx, "f1"] = mean_f1
        df_new.at[idx, "f2"] = mean_f2
        df_new.at[idx, "f3"] = mean_f3
        df_new.at[idx, "f4"] = mean_f4
        df_new.at[idx, "f5"] = mean_f5
        df_new.at[idx, "f6"] = mean_f6
        df_new.at[idx, "similar_brand_taste"] = "|".join(group_similar_taste)
            
    return df_new

In [11]:
df_filling = filling_data(df_initial, df_missing_f)

In [12]:
df_f_clean = df_filling[df_filling.f1.notnull()].reset_index()

In [13]:
df_f_clean.shape

(829, 20)

## Handle duplicate values

In [14]:
df_duplicates = df_f_clean[df_f_clean.name.duplicated() | df_f_clean.f1.duplicated()]

In [15]:
df_duplicates

,index,name,intl_name,brand_name,brand_intl_name,year_month,rank,score,f1,f2,f3,f4,f5,f6,flavour_tags,checkin_count,pictures,similar_brands,id,similar_brand_taste
13,15,千歳鶴,Chitosetsuru,北海道,Hokkaido,202001,16,4.008530,0.354254,0.175449,0.326109,0.385708,0.525363,0.559105,|甘辛い|昔ながら|さらり|ラム|砂糖|栗|レモン|上品|清涼|辛口|じっくり|チョコレート...,83,https://s3-ap-northeast-1.amazonaws.com/physal...,るみ子の酒|久保田|正雪|上善如水|銀盤,3679,千歳鶴|常山|渓|大盃|天青|南|白鷹|臥龍梅|青煌|七冠馬
45,58,せきのいち,Sekinoichi,岩手県,Iwate,202001,21,4.006629,0.345280,0.433808,0.407720,0.484783,0.368802,0.394116,|ナッツ|アミノ酸|薄い|熱燗|控えめ|芳醇|苦味|濃厚|スッキリ|フルーティ|酸味|しっか...,34,https://s3-ap-northeast-1.amazonaws.com/physal...,富久駒|有薫|香の泉|紗利|鯨波,1134,龍力|松の寿|玉乃光|御前酒|桃の滴|環日本海|会津中将|天の戸|山形正宗|弥右衛門
66,83,墨廼江,Suminoe,宮城県,Miyagi,202001,19,4.007084,0.438280,0.362267,0.248958,0.345579,0.417966,0.542793,|花|柑橘|あっさり|バランス|リンゴ|キレ|スッキリ|フレッシュ|辛口|さっぱり|さわやか...,42,https://s3-ap-northeast-1.amazonaws.com/physal...,あずまみね|酔鯨|羽陽男山|大信州|北の勝,1106,陸奥八仙|宮泉|櫛羅|菊盛|山本|総乃寒菊|黒兜|高千代|三好|澤の花|赤武|鳥海山|紀土|...
70,89,浦霞,Urakasumi,宮城県,Miyagi,202001,25,4.006880,0.422565,0.075997,0.041048,0.172511,0.619512,0.632789,|芳香|砂糖|スッキリ|爽快|さらり|辛口|心地よい|豊か|綺麗|コク|スイスイ|とろみ|リ...,104,https://s3-ap-northeast-1.amazonaws.com/physal...,大信州|れいざん|乾坤一|楯野川|久保田,1107,浦霞|笹祝|ど|繁枡|ひと夏の恋|永寳屋|寒北斗|田中六十五|土佐鶴|鳳陽
96,122,花邑,Hanamura,秋田県,Akita,202001,18,4.006616,0.604991,0.286818,0.147817,0.300620,0.388021,0.537530,|華やか|甘味|綺麗|硬い|蜜|スッキリ|グレープフルーツ|ピリリ|花|安定|マスカット|柔...,122,https://s3-ap-northeast-1.amazonaws.com/physal...,十四代|紀土|花の香|雨後の月|作,1008,花邑|水芭蕉|日榮|甲子正宗|雅山流|澤の花|信州亀齢|越後桜|夜明け前|奥丹波
106,132,鳥海山,Chokaisan,秋田県,Akita,202001,28,4.006602,0.409185,0.426256,0.325100,0.315681,0.438497,0.503239,|レーズン|爽快|フルーティ|レモン|辛口|さわやか|旨味|穀物|ちびちび|スッキリ|控えめ...,84,https://s3-ap-northeast-1.amazonaws.com/physal...,紀土|酔鯨|七田|山形正宗|陸奥八仙,1095,鳥海山|紀伊国屋文左衛門|百十郎|雨後の月|高千代|早瀬浦|手取川|秀鳳|大山|田光
153,196,辯天,Benten,山形県,Yamagata,202001,44,4.006602,0.406718,0.372218,0.264269,0.402433,0.448167,0.490173,|チョコレート|安定|柑橘|あっさり|バランス|ピリリ|フルーティ|控えめ|濃厚|スッキリ|...,37,https://s3-ap-northeast-1.amazonaws.com/physal...,八海山|飛露喜|田酒|雪の茅舎|獺祭,168,紀土|黒龍|結ゆい|加茂錦|出羽桜|奈良萬|五橋|磯自慢|あべ|上川大雪|初孫|宮の雪|榮川...
197,243,宮泉,Miyaizumi,福島県,Fukushima,202001,41,4.006608,0.584340,0.449256,0.305877,0.220966,0.303023,0.471706,|桃|フルーティ|ずっしり|じわじわ|酸味|キャラメル|ガス|まろやか|華やか|メロン|なめ...,59,https://s3-ap-northeast-1.amazonaws.com/physal...,尾瀬の雪どけ|冬の月|吉田蔵|口万|央,1391,宮泉|燦爛|亀甲花菱|開華|黒兜|醸し人九平次|栄光冨士|花の香|三好|山の壽
201,248,ゆり,Yuri,福島県,Fukushima,202001,46,4.006605,0.381456,0.460119,0.441871,0.473519,0.278715,0.381615,|スイスイ|渋み|ガス|キリリ|しっかり|柔らかい|スッキリ|苦味|綺麗|バランス|まろやか...,64,https://s3-ap-northeast-1.amazonaws.com/physal...,三十六人衆|神月|亀の井|相模灘|登龍,1090,梵|遊穂|花泉|会津中将|豊久仁|伝心|村祐|奥|春霞|千福
217,268,副将軍,Fukushogun,茨城県,Ibaraki,202001,16,4.008232,0.452536,0.408454,0.262259,0.383264,0.363625,0.511401,|ちびちび|冷酒|さわやか|ガス|キリリ|苦味|フレッシュ|バランス|まろやか|酸味|スッキ...,28,https://s3-ap-northeast-1.amazonaws.com/physal...,十右衛門|澤屋まつもと|残草蓬莱|月の井|大嶺,242,陸奥八仙|宮泉|櫛羅|菊盛|山本|総乃寒菊|黒兜|高千代|三好|澤の花|雪の茅舎|羽根屋|信...


In [16]:
df_dup_cleaned = df_f_clean.drop_duplicates(subset=["f1"], keep="last")

In [17]:
df_dup_cleaned = df_dup_cleaned.reset_index()

In [18]:
# df_dup_cleaned[df_dup_cleaned.f6|.duplicated()]

## Implement the algorithm

In [19]:
flavour_matrix = df_dup_cleaned.as_matrix(flavour_columns)

In [32]:
flavour_matrix

array([[0.51610285, 0.48476329, 0.20302933, 0.41948098, 0.2760852 ,
        0.45302778],
       [0.18775025, 0.30485848, 0.41715547, 0.53993487, 0.52749097,
        0.3577933 ],
       [0.27760726, 0.39443511, 0.3270922 , 0.52083516, 0.45426989,
        0.42747563],
       ...,
       [0.34830165, 0.50601983, 0.35353431, 0.40070638, 0.43424293,
        0.38537109],
       [0.21894512, 0.40032345, 0.35982794, 0.66737753, 0.32397476,
        0.33462754],
       [0.33258599, 0.50968856, 0.3142955 , 0.47003514, 0.42471331,
        0.35987225]])

## GMM

In [21]:
n_clusters = 50

In [52]:
class GMM2D:
    def __init__(self, data, n_clusters=50, max_iterators=200):
        self.n_clusters = n_clusters
        self.max_iterators = max_iterators
        self.data = data * 10
        self.eps = 1e-6
    
    def init_params(self):
        weight_init = np.ones((self.n_clusters)) / self.n_clusters
        mean_init = np.random.choice(self.data.flatten(), size=(self.n_clusters, self.data.shape[1]))
        co_variance = []

        for i in range(self.n_clusters):
            co_variance.append(make_spd_matrix(self.data.shape[1]))

        co_variance = np.array(co_variance)
        
        return weight_init, mean_init, co_variance
    
    def gaussian_density_function(self, data, mean, variance):
        f1 = 1/math.sqrt(2*math.pi*variance)
        f2 = math.exp(-(data - mean)**2/(2*variance))

        return f1*f2
    
    def run(self):
        weight_init, mean_init, co_variance = self.init_params()
        x = np.linspace(np.min(self.data[...,0])-1, np.max(self.data[...,0])+1, 100)
        y = np.linspace(np.min(self.data[...,1])-1, np.max(self.data[...,1])+1, 80)
        X_, Y_ = np.meshgrid(x, y)
        pos = np.array([X_.flatten(), Y_.flatten()]).T
        colors = ['tab:blue', 'tab:orange', 'tab:green', 'magenta', 'yellow', 'red', 'brown', 'grey']
        
        for step in range(self.max_iterators):
#             if step%1 == 0:
#                 plt.figure(figsize=(12, int(8)))
#                 plt.title("Interator -- {}".format(step))
#                 axes = plt.gca()
                
#                 likelihood = []
#                 for j in range(self.n_clusters):
#                     likelihood.append(multivariate_normal.pdf(x=pos, mean=mean_init[j][:2], cov=co_variance[j][: ,:2]))
                    
#                 likelihood = np.array(likelihood)
#                 predictions = np.argmax(likelihood, axis=0)
#                 print(predictions)
                
#                 for c in range(k):
#                     pred_ids = np.where(predictions == c)
#                     plt.scatter(pos[pred_ids[0],0], pos[pred_ids[0],1], color=colors[c], alpha=0.2, edgecolors='none', marker='s')
    
#                 plt.scatter(X[...,0], X[...,1], facecolors='none', edgecolors='grey')
    
#                 for j in range(k):
#                     plt.scatter(means[j][0], means[j][1], color=colors[j])
                
#                 plt.show()
#             print(mean_init.shape, weight_init.shape, co_variance.shape)

            likelihood = []
            for j in range(self.n_clusters):
                likelihood.append(multivariate_normal.pdf(x=self.data, mean=mean_init[j], cov=co_variance[j]))
                
            likelihood = np.array(likelihood)            
            
            b = []
            for j in range(self.n_clusters):
                b.append((likelihood[j]*weight_init[j]) / (np.sum([likelihood[i]*weight_init[i] for i in range(self.n_clusters)], axis=0)+self.eps))
                mean_init[j] = np.sum(b[j].reshape(len(self.data), 1)*self.data, axis=0) / (np.sum(b[j]+self.eps))
                co_variance[j] = np.dot((b[j].reshape(len(self.data), 1)*(self.data - mean_init[j])).T, (self.data - mean_init[j])) / (np.sum(b[j]+self.eps))
                weight_init[j] = np.mean(b[j])
                
                if np.linalg.det(co_variance[j]) < 1e-5:
                    continue
#                     mean_init[j] = mean_init[j-1]
#                     co_variance[j] = co_variance[j-1]
#                     weight_init[j] = weight_init[j-1]
                print("step----{}--j-----{}".format(step, j))
#                 print(multivariate_normal.pdf(x=self.data, mean=mean_init[j], cov=co_variance[j]))
                print(np.linalg.det(co_variance[j]))
                

In [53]:
gmm_2d = GMM2D(flavour_matrix, max_iterators=10)

In [54]:
gmm_2d.run()

step----0--j-----1
0.0020619506752901856
step----0--j-----4
0.00035204476352407223
step----0--j-----5
0.0007325041631157614
step----0--j-----7
6.836483377221774e-05
step----0--j-----8
7.090546708885364e-05
step----0--j-----9
0.00022993774974493323
step----0--j-----10
2.7597553311332523e-05
step----0--j-----11
1.0068027028543807e-05
step----0--j-----12
0.00033776873631855946
step----0--j-----13
0.0005294937042614736
step----0--j-----15
0.0001425711508565644
step----0--j-----16
7.381430924964502e-05
step----0--j-----18
0.0011788747809791517
step----0--j-----19
0.0007848731101353208
step----0--j-----20
9.00954972243562e-05
step----0--j-----21
0.0013846752322439666
step----0--j-----22
0.00010252576277393733
step----0--j-----23
8.246878771717947e-05
step----0--j-----24
0.00043987571898578963
step----0--j-----26
8.567913239082243e-05
step----0--j-----28
2.0586638953872765e-05
step----0--j-----31
0.00011665154185705783
step----0--j-----32
0.0007610209289871317
step----0--j-----34
1.4723957838

LinAlgError: singular matrix